In [1]:
%load_ext autoreload
%autoreload 2
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from os import listdir, makedirs

from pin_drop_sampling2.utils import gen_rooftop_map

During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 


In [2]:
# Set paths and paramters
PREPROCESSED_PATH = Path("../data/03_Output/Kilifi -- Kenya/01_filtered_shapes")
SAMPLES_PATH = Path("../data/03_Output/Kilifi -- Kenya/02_sampled_locations_and_rooftops")
OUTPUT_PATH = Path("../data/03_Output/Kilifi -- Kenya/03_maps_and_kml_files")

# Load files / params
subcounties = listdir(PREPROCESSED_PATH)

In [24]:
for subcounty in subcounties:
    # Load data
    sampled_sublocations = gpd.read_parquet(SAMPLES_PATH / subcounty / "sampled_sublocations.parquet")
    # Save kml
    makedirs(OUTPUT_PATH / subcounty, exist_ok=True)
    sampled_sublocations.to_file(OUTPUT_PATH / subcounty / "sampled_sublocations.kml", driver="KML")

    makedirs(OUTPUT_PATH / subcounty / "maps", exist_ok=True)
    makedirs(OUTPUT_PATH / subcounty / "kml_files", exist_ok=True)
    makedirs(OUTPUT_PATH / subcounty / "gmap_links", exist_ok=True)

    for i, (_, sublocation) in enumerate(sampled_sublocations.iterrows()):
        sl_name = sublocation.SLNAME.replace("/", "-")
        # Load rooftops
        rooftops = gpd.read_parquet(
            SAMPLES_PATH / 
            subcounty / 
            f"{i+1}_{sl_name}_sampled_rooftops.parquet")
        rooftops.geometry = gpd.points_from_xy(rooftops.centroid.x, rooftops.centroid.y)
        # rooftops = rooftops.set_crs(epsg=4326)
        map = gen_rooftop_map(sublocation.geometry, 
                              rooftops)
        map.save(str(OUTPUT_PATH / subcounty / "maps" / f"{i+1}_{sl_name}_rooftop_map.html"))
        rooftops.to_file(OUTPUT_PATH / subcounty / "kml_files" / f"{i+1}_{sl_name}_rooftops.kml", driver="KML")

        # Create df with gmap links
        df = pd.DataFrame(
            {
                "subcounty": [subcounty] * len(rooftops),
                "sublocation": [sublocation.SLNAME] * len(rooftops),
                "rooftop_id": range(1, len(rooftops) + 1),
                "gmap_link": [f"https://www.google.com/maps/search/?api=1&query={row.geometry.centroid.y},{row.geometry.centroid.x}"
                              for _, row in rooftops.iterrows()]
            }
        )
        df.to_csv(OUTPUT_PATH / subcounty / "gmap_links" / f"{i+1}_{sl_name}_gmap_links.csv", index=False)





/Users/poornimaramesh/Library/Caches/pypoetry/virtualenvs/pin-drop-sampling2-RChTn2Ku-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Value 'KAMBE/RIBE' of field sampled_sublocations.share parsed incompletely to real 0.
  ogr_write(
/Users/poornimaramesh/Library/Caches/pypoetry/virtualenvs/pin-drop-sampling2-RChTn2Ku-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Lossy conversion occurred when trying to set 64 bit integer field from a real value.
  ogr_write(
/Users/poornimaramesh/Library/Caches/pypoetry/virtualenvs/pin-drop-sampling2-RChTn2Ku-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Value 'Exact' of field sampled_sublocations.density parsed incompletely to real 0.
  ogr_write(
/Users/poornimaramesh/Library/Caches/pypoetry/virtualenvs/pin-drop-sampling2-RChTn2Ku-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Value 'RABAI' of field sampled_sublocations.share parsed incompletely to re